# Story - 2 : Can the FED Control Inflation and Maintain Full Employment

### Instructions
The Federal Reserve's mandate from Congress is to control inflation and to maintain low unemployment. These seem to be contradictory objectives. 
For this story you will need to source the following data for the last 25 years; 
 - The Consumer Price Index (CPI) (Bureau of Labor Statistics)
 - The FED Funds Rate (FRED) (Federal Reserve Board)
 - Unemployment Rate  (Bureau of Labor Statistics)

Your Data Visualizations should be designed to answer the question "Has the FED been able to fulfill the mandate given to it by Congress?"

Notes:
- You must use a different application or library to create your visualization(s) from what you used for Story -1 (e.g. If you used Excel for Story -1 you may not use it for this story)
- You will receive 20 bonus point if you access the data through available APIs using code that you develop
- Remember, the FED raises rate after reviewing the CPI and other data and unemployment (layoffs) occur after company operating costs go up.
- This assignment is due at the end of week four of the semester

### Story 2 Notes:

- For Story 1, I made use of matplotlib, for this one i will use plotly as per the requirement of leveraging a different tool/ application. If this isnt sufficient please let me know and i can do in R.
- Leveraging the APIs for pulling in data for bonus points.

In [1]:
import pandas as pd 
import os
import numpy as np
import plotly as pt
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'notebook'
import requests
import json
from datetime import date

In [2]:
### Establishing Environmental Credentials
home_directory = os.path.expanduser("~")
cred_file = os.path.join(home_directory , 'my_creds.json')
with open(cred_file, 'r') as f:
    creds = json.load(f)
bls_key = creds['BLS']["api_key"]
fred_key = creds['fred']["api_key"]

#### Fed Reserve Data 

In [3]:
series_id = "FEDFUNDS"  # or use "DFF" for daily
base_url = "https://api.stlouisfed.org/fred/series/observations"
params = {"series_id": "FEDFUNDS",
          "api_key": fred_key,
           "file_type": "json",
          "observation_start": "1999-01-01"}
response = requests.get(base_url, params=params, timeout=30)
response.raise_for_status()
data = response.json()
fed_res_df = pd.DataFrame(data["observations"])
fed_res_df["date"] = pd.to_datetime(fed_res_df["date"])
fed_res_df = fed_res_df[["date","value"]].rename(columns={"value":"fed_funds_rate"})

#### BLS Data 
##### Needs two pulls for each data setbecause limits on API, the first pull stops at 2019. need to do second pull for each from 2020 throuh 2025


In [4]:
## Unemployment pulls
base_url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
series_id = "LNS14000000" #https://data.bls.gov/timeseries/LNS14000000
#CEU0800000003 #https://www.bls.gov/help/hlpforma.htm#CE
payload = {
    "registrationKey": bls_key,
    "seriesid": [series_id],
    "startyear": "1999",
    "endyear": str(date.today().year)
}
response = requests.post(base_url, json=payload)
data = response.json()
bls_unemp = pd.DataFrame(data["Results"]["series"][0]["data"])
bls_unemp["date"] = pd.to_datetime(bls_unemp["year"] + "-" + bls_unemp["period"].str[1:], format="%Y-%m")
bls_unemp = bls_unemp[["date",'value']].rename(columns={"value":"unemployment_rate"})
if bls_unemp['date'].max().year<2025:
    print(bls_unemp['date'].max().date()," is max date")
    print("need second pull. Starting that now.")
    ## Unemployment 2
    payload = {
        "registrationKey": bls_key,
        "seriesid": [series_id],
        "startyear": str(bls_unemp['date'].max().year),
        "endyear": str(date.today().year)
    }
    response = requests.post(base_url, json=payload)
    data = response.json()
    bls_unemp2= pd.DataFrame(data["Results"]["series"][0]["data"])
    bls_unemp2["date"] = pd.to_datetime(bls_unemp2["year"] + "-" + bls_unemp2["period"].str[1:], format="%Y-%m")
    bls_unemp2 = bls_unemp2[["date",'value']].rename(columns={"value":"unemployment_rate"})
    bls_unemp = pd.concat([bls_unemp,bls_unemp2]).drop_duplicates()
#confirming content
if bls_unemp['date'].max().year == date.today().year:
    print ("date is recent")
else:
    print("Please check the pulls")

2018-12-01  is max date
need second pull. Starting that now.
date is recent


In [5]:
## CPI Data 
## Unemployment
base_url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
series_id = "CUSR0000SA0" 
#CEU0800000003 #https://www.bls.gov/help/hlpforma.htm#CE
payload = {
    "registrationKey": bls_key,
    "seriesid": [series_id],
    "startyear": "1999",
    "endyear": str(date.today().year)
}
response = requests.post(base_url, json=payload)#, timeout=30)
data = response.json()
bls_cpi = pd.DataFrame(data["Results"]["series"][0]["data"])
bls_cpi["date"] = pd.to_datetime(bls_cpi["year"] + "-" + bls_cpi["period"].str[1:], format="%Y-%m")
bls_cpi = bls_cpi[["date",'value']].rename(columns={"value":"cpi_value"})
if bls_cpi['date'].max().year<2025:
    print(bls_cpi['date'].max().date()," is max date")
    print("need second pull. Starting that now.")
    payload = {
        "registrationKey": bls_key,
        "seriesid": [series_id],
        "startyear": str(bls_cpi['date'].max().year),
        "endyear": str(date.today().year)
    }
    response = requests.post(base_url, json=payload)#, timeout=30)
    data = response.json()
    bls_cpi2 = pd.DataFrame(data["Results"]["series"][0]["data"])
    bls_cpi2["date"] = pd.to_datetime(bls_cpi2["year"] + "-" + bls_cpi2["period"].str[1:], format="%Y-%m")
    bls_cpi2 = bls_cpi2[["date",'value']].rename(columns={"value":"cpi_value"})
    bls_cpi = pd.concat([bls_cpi,bls_cpi2]).drop_duplicates()
#confirming content
if bls_cpi['date'].max().year == date.today().year:
    print ("date is recent")
else:
    print("Please check the pulls")

2018-12-01  is max date
need second pull. Starting that now.
date is recent


#### Puting into one working df

In [6]:
## Putting all the information into one df 
story_2_data = fed_res_df.merge(bls_unemp, how="outer", on=["date"]).merge(bls_cpi,how="outer", on=["date"])
story_2_data = story_2_data.set_index('date')

## Making the values numbers
story_2_data["fed_funds_rate"] = pd.to_numeric(story_2_data["fed_funds_rate"])
story_2_data["unemployment_rate"] = pd.to_numeric(story_2_data["unemployment_rate"])
story_2_data["cpi_value"] = pd.to_numeric(story_2_data["cpi_value"])

## Calculating Inflation (Year over Year Inflation using CPI value changes) 
story_2_data['inflation_rate'] = round(((story_2_data['cpi_value'] - story_2_data['cpi_value'].shift(12)) / story_2_data['cpi_value'].shift(12))  * 100,2)

## Trimming where no Inflation Rate Value (Removing year 1999 basically)
story_2_data = story_2_data[~story_2_data["inflation_rate"].isnull()]

## Putting back for now
story_2_data = story_2_data.reset_index()

## Renaming COlumns for Proper Labeling 
story_2_data = story_2_data.rename(columns={"unemployment_rate":"Unemployment", 
                                           "fed_funds_rate":"Fed Funds",
                                           "inflation_rate":"Inflation"})

### Adding Quarterly Timestamp into DF 
story_2_data['year'] = story_2_data['date'].dt.year
story_2_data['quarter']= story_2_data['date'].dt.quarter
story_2_data['yr_qtr'] = story_2_data['date'].dt.to_period('Q').astype(str)
story_2_data['half'] = np.where(story_2_data['date'].dt.month <= 6, 'H1', 'H2')
story_2_data['yr_half'] = (story_2_data['date'].dt.year.astype(str) + ' ' + story_2_data['half'])
story_2_data.head(n=5)

,date,Fed Funds,Unemployment,cpi_value,Inflation,year,quarter,yr_qtr,half,yr_half
0,2000-01-01,5.45,4.0,169.3,2.79,2000,1,2000Q1,H1,2000 H1
1,2000-02-01,5.73,4.1,170.0,3.22,2000,1,2000Q1,H1,2000 H1
2,2000-03-01,5.85,4.0,171.0,3.76,2000,1,2000Q1,H1,2000 H1
3,2000-04-01,6.02,3.8,170.9,3.01,2000,2,2000Q2,H1,2000 H1
4,2000-05-01,6.27,4.0,171.2,3.13,2000,2,2000Q2,H1,2000 H1


### Begining of Visualization work.

In [31]:

fig = px.line(
    story_2_data,
    x='date',
    y=['Fed Funds', 'Unemployment', 'Inflation'],
    title="The Fed's Mandate is to control inflation, the year over year change in CPI, and to maintain low unemployment. <br> Firstly, lets take a look at the fed funds rate, unemployment, and inflation for the past 25 years.<br> So, does the Fed adhere to this mandate?",
    labels={'date': '', 'variable': '',"value":"Rate (%)"},
    color_discrete_sequence= ['#002D4C', '#7B0323', '#E89600'] 
)
fig.update_traces(line=dict(width=3.3),    hovertemplate='%{y:.2f}%<extra></extra>')
fig.update_layout(paper_bgcolor='white',plot_bgcolor='white',hovermode='x unified', 
    legend=dict(orientation='h', yanchor='top', y=-0.25,xanchor='center', x=0.5, font=dict(size=17, color='#2F4F4F')),
    title=dict(font=dict(size=17, color='#2F4F4F'), x=0.5, xanchor='center'))

fig.update_xaxes( tickfont=dict(color='#2F4F4F', size=14,))
fig.update_yaxes( tickfont=dict(color='#2F4F4F', size=14,),    
                 title_font=dict(color='#2F4F4F', size=18), 
                 ticklabelposition="outside", ticklabelstandoff=12)
fig.show()

In [8]:
# --- DID NOT USE ----

### Net Change Quarterly In percent change 
# quarterly_net_change = story_2_data.groupby('yr_qtr').agg({'Fed Funds': lambda x: round(((x.iloc[-1] - x.iloc[0])/x.iloc[0])*100,2),
#                                                            'Unemployment': lambda x: round(((x.iloc[-1] - x.iloc[0])/x.iloc[0])*100,2),
#                                                            'Inflation': lambda x: round(((x.iloc[-1] - x.iloc[0])/x.iloc[0])*100,2),}).reset_index()

# # Add shifted columns for unemployment and inflation (next quarter's values)
# quarterly_net_change['Unemployment Proceeding'] = quarterly_net_change['Unemployment'].shift(-1)
# quarterly_net_change['Inflation Proceeding'] = quarterly_net_change['Inflation'].shift(-1)
# quarterly_net_change['Half Year Proceeding'] = quarterly_net_change['yr_qtr'].shift(-1)
# quarterly_net_change.head(5)

In [9]:
### Further processing for later Charts
### Net Change half years In percent change 
half_net_change = story_2_data.groupby('yr_half').agg({'Fed Funds': lambda x: round(((x.iloc[-1] - x.iloc[0])/x.iloc[0])*100,2),
                                                           'Unemployment': lambda x: round(((x.iloc[-1] - x.iloc[0])/x.iloc[0])*100,2),
                                                           'Inflation': lambda x: round(((x.iloc[-1] - x.iloc[0])/x.iloc[0])*100,2),}).reset_index()

# Add shifted columns for unemployment and inflation (next quarter's values)
half_net_change['Unemployment Proceeding'] = half_net_change['Unemployment'].shift(-1)
half_net_change['Inflation Proceeding'] = half_net_change['Inflation'].shift(-1)
half_net_change['Half Year Proceeding'] = half_net_change['yr_half'].shift(-1)
half_net_change.head(5)

,yr_half,Fed Funds,Unemployment,Inflation,Unemployment Proceeding,Inflation Proceeding,Half Year Proceeding
0,2000 H1,19.82,0.00,33.69,-2.50,-4.44,2000 H2
1,2000 H2,-2.14,-2.50,-4.44,7.14,-14.25,2001 H1
2,2001 H1,-33.61,7.14,-14.25,23.91,-41.18,2001 H2
3,2001 H2,-51.72,23.91,-41.18,1.75,-10.83,2002 H1
4,2002 H1,1.16,1.75,-10.83,3.45,68.71,2002 H2


In [32]:
fig = px.bar(half_net_change, x='yr_half', y=['Fed Funds', 'Unemployment Proceeding'],    
             title="""Assuming a six month lag in the response time of the economy, a change in the Fed Funds Rate <br>
             generally seems to lead to a decrease in the unemployment rate for the proceeding six months.""",
    color_discrete_map={'Fed Funds': '#002D4C','Unemployment Proceeding': '#7B0323'},
    labels={'value': 'Percent Change', 'yr_half': '', 'variable': '',},)
fig.update_layout(#width=1200,height=500,
    hovermode='x unified', 
    barmode='group',paper_bgcolor='white', plot_bgcolor='white',bargap=0.5,   
    legend=dict(orientation='h', yanchor='top', y=-0.25,xanchor='center', x=0.5, font=dict(size=17, color='#2F4F4F')),
    title=dict(font=dict(size=17, color='#2F4F4F'), x=0.5, xanchor='center'))
fig.update_xaxes(tickfont=dict(color='#2F4F4F', size=14,),showticklabels=False)
fig.update_yaxes(range=[-150, 150],
                 zeroline=True,  
                 zerolinecolor='#2F4F4F',
                 zerolinewidth=0.5,
                 tickfont=dict(color='#2F4F4F', size=15,),    
                 title_font=dict(color='#2F4F4F', size=18),)
fig.for_each_trace(lambda trace: trace.update(name='Federal Funds Rate Change' if trace.name == 'Fed Funds' else "Unemployment Change (Following 6 Months)"))
fig.update_traces(customdata=half_net_change[['Unemployment Proceeding', 'Half Year Proceeding']].to_numpy(),
                  hovertemplate=( "<span style='font-size:12px;'>"
                                 "The Fed Funds Rate change was %{y:.2f}%."
                                 "</span><extra></extra>"),
    selector=lambda t: t.name.lower().startswith('fed') or t.name.lower().startswith('federal')
)
fig.update_traces(customdata=half_net_change[['Unemployment Proceeding', 'Half Year Proceeding']].to_numpy(),
                  hovertemplate=( "<span style='font-size:12px;'>"
                                 "This yielded a %{customdata[0]:.2f}% change in the unemployment rate "
                                 "in %{customdata[1]}."
                                 "</span><extra></extra>"),
    selector=lambda t: t.name.lower().startswith('unem') or t.name.lower().startswith('unemployment')
)
fig.show()

In [11]:
viz3 =  half_net_change[half_net_change['Fed Funds']!=0]
viz3["Reduced Unemployment"] = 'Increased or Maintained Unemployment'
viz3["Reduced Unemployment"][viz3["Unemployment Proceeding"]<0] = 'Decreased Unemployment'
viz_3_grouped = viz3.groupby(["Reduced Unemployment"]).agg({"yr_half":"count"}).reset_index()#.rename(columns={})
viz_3_grouped 

C:\Users\johnf\AppData\Local\Temp\ipykernel_31300\1548246860.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\johnf\AppData\Local\Temp\ipykernel_31300\1548246860.py:3: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to per

,Reduced Unemployment,yr_half
0,Decreased Unemployment,24
1,Increased or Maintained Unemployment,18


In [17]:
fig = px.pie(    viz_3_grouped,
    names='Reduced Unemployment',
    values='yr_half',
    color='Reduced Unemployment',
    color_discrete_map={'Increased or Maintained Unemployment': '#D3D3D3', 'Decreased Unemployment': '#7B0323'},
    title=f"""Of the {viz3.shape[0]} times that the feds fund rate was changed, so action was taken by the fed in an attempt<br>
    to further its mandate a clear majority of instances resulted in a decrease in unemployement in the following 6 months.""")
fig.update_layout(legend=dict(orientation='h', yanchor='top', y=-0.25,xanchor='center', x=0.5, font=dict(size=17, color='#2F4F4F')),
                  title=dict(font=dict(size=17, color='#2F4F4F'), x=0.5, xanchor='center'))
fig.update_traces(textinfo='none',
                  #textfont_size=20,
                  pull=[0, 0.08],
                  hovertemplate="%{label}: %{value} occurrences (%{percent})<extra></extra>",
                  textfont=dict(size=22, color='white'),)
fig.show()

In [33]:
fig = px.bar(half_net_change, x='yr_half', y=['Fed Funds', 'Inflation Proceeding'],    
             title="""Here is the same chart looking at 6 month percent changes in the Fed Funds rate, but this time looking at inflation.<br> Inflation, unlike unemployment, is different. While lower unemployment is always the goal, the Federal reserve<br> 
             has an inflation goal of 2%.<br> """,
    color_discrete_map={'Fed Funds': '#002D4C','Inflation Proceeding': '#E89600'},
    labels={'value': 'Percent Change', 'yr_half': '', 'variable': '',},)
fig.update_layout(#width=1200,height=500,
    hovermode='x unified', 
    barmode='group',paper_bgcolor='white', plot_bgcolor='white',bargap=0.5,   
    legend=dict(orientation='h', yanchor='top', y=-0.25,xanchor='center', x=0.5, font=dict(size=17, color='#2F4F4F')),
    title=dict(font=dict(size=17, color='#2F4F4F'), x=0.5, xanchor='center'))
fig.update_xaxes(tickfont=dict(color='#2F4F4F', size=14,),showticklabels=False)
fig.update_yaxes(range=[-150, 150],
                 zeroline=True,  
                 zerolinecolor='#2F4F4F',
                 zerolinewidth=0.5,
                 tickfont=dict(color='#2F4F4F', size=15,),    
                 title_font=dict(color='#2F4F4F', size=18),)
fig.for_each_trace(lambda trace: trace.update(name='Federal Funds Rate Change' if trace.name == 'Fed Funds' else "Inflation Change (Following 6 Months)"))
fig.update_traces(#customdata=half_net_change[['Inflation Proceeding', 'Half Year Proceeding']].to_numpy(),
                  hovertemplate=( "<span style='font-size:12px;'>"
                                 "The Fed Funds Rate change was %{y:.2f}%."
                                 "</span><extra></extra>"),
    selector=lambda t: t.name.lower().startswith('fed') or t.name.lower().startswith('federal')
)
fig.update_traces(
    customdata=half_net_change[['Inflation Proceeding','Half Year Proceeding']].to_numpy(),
    hovertemplate=("<span style='font-size:12px;'>"
                   "The inflation change (following 6 months) was %{y:.2f}%<br>"
                   "for %{customdata[1]}."
                   "</span><extra></extra>"
    ),
    selector=lambda t: 'inflation' in t.name.lower()          # <-- key change
)
fig.show()

In [14]:
viz5=  half_net_change[half_net_change['Fed Funds']!=0]
mid1 = story_2_data[["yr_half","Inflation"]].groupby(["yr_half"]).agg({'Inflation': lambda x:  x.iloc[0]}).reset_index().rename(columns={"Inflation":"inflation_start"})
mid2 = story_2_data[["yr_half","Inflation"]].groupby(["yr_half"]).agg({'Inflation': lambda x:  x.iloc[-1]}).reset_index().rename(columns={"Inflation":"inflation_end"})
viz5_part1 = mid1.merge(mid2,how='inner',on='yr_half')
viz5_part1['closer_to_goal'] = (abs(viz5_part1['inflation_end'] - 2) < abs(viz5_part1['inflation_start'] - 2))
viz_5_final = viz5[["yr_half","Fed Funds","Half Year Proceeding"]].merge(viz5_part1[["yr_half","closer_to_goal"]],
                                                           left_on=["Half Year Proceeding"],
                                                           right_on=["yr_half"],
                                                           how='inner')
viz_5_final = viz_5_final.rename(columns={"yr_half_x":"yr_half","closer_to_goal":"closer_to_goal_proceeding"}).drop(columns=["yr_half_y"])
viz_5_final.head()

,yr_half,Fed Funds,Half Year Proceeding,closer_to_goal_proceeding
0,2000 H1,19.82,2000 H2,True
1,2000 H2,-2.14,2001 H1,True
2,2001 H1,-33.61,2001 H2,True
3,2001 H2,-51.72,2002 H1,False
4,2002 H1,1.16,2002 H2,True


In [15]:
viz5_grouped = viz_5_final.groupby("closer_to_goal_proceeding").agg({'yr_half':'count'}).reset_index()
viz5_grouped["closer_to_goal_proceeding"][viz5_grouped["closer_to_goal_proceeding"]==True] = "Closer to 2% Goal"
viz5_grouped["closer_to_goal_proceeding"][viz5_grouped["closer_to_goal_proceeding"]==False] = "Further from 2% Goal"
viz5_grouped

C:\Users\johnf\AppData\Local\Temp\ipykernel_31300\763157255.py:2: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


C:\Users\johnf\AppData\Local\Temp\ipykernel_31300\763157255.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy

,closer_to_goal_proceeding,yr_half
0,Further from 2% Goal,17
1,Closer to 2% Goal,25


In [30]:
fig = px.pie(viz5_grouped,
    names='closer_to_goal_proceeding',
    values='yr_half',
    color='closer_to_goal_proceeding',
    color_discrete_map={'Further from 2% Goal': '#D3D3D3', 'Closer to 2% Goal': '#E89600'},
    title=f"""Of the {viz5.shape[0]} times that the feds fund rate was changed, so action was taken by the fed in an attempt<br>
    to achieve its mandate a clear majority of instances resulted in a movement toward the 2% inflation goal<br> in the following 6 months."""
            )
fig.update_layout(legend=dict(orientation='h', yanchor='top', y=-0.25,xanchor='center', x=0.5, font=dict(size=17, color='#2F4F4F')),
                  title=dict(font=dict(size=17, color='#2F4F4F'), x=0.5, xanchor='center'))
fig.update_traces(textinfo='none',
                  #textfont_size=20,
                  pull=[0, 0.08],
                  hovertemplate="%{label}: %{value} occurrences (%{percent})<extra></extra>",
                  textfont=dict(size=22, color='white'),)
fig.show()

#### Overall, based on the rough 60% success rate that the Federal Reserve has for the dual mandate - to keep inflation in check, and unemployment low - one can conclude that the Federal Reserve is satisfactorily meeting its mandate.